In [1]:
#!conda install -c conda-forge geopy --yes
#!conda install -y lxml --yes
import numpy as np
import pandas as pd
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
print('Libraries imported.')

Libraries imported.


In [2]:
web = pd.read_html('http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
neighborhoods = web[0]
neighborhoods.dropna
neighborhoods = neighborhoods[neighborhoods.Borough != 'Not assigned']
neighborhoods = neighborhoods.reset_index(drop=True)
a = neighborhoods.where(neighborhoods.Neighbourhood == 'Not assigned')
a = a.dropna()
idx = a.index.values

for i in range(len(idx)):
    neighborhoods.iloc[idx[i]]['Neighbourhood'] = neighborhoods.iloc[idx[i]]['Borough']

neighborhoods = neighborhoods.sort_values(by=['Postcode'])
neighborhoods = neighborhoods.reset_index(drop=True)
dx = neighborhoods.index.values

for i in range(1, len(dx)):
    if neighborhoods.iloc[dx[i]]['Postcode'] == neighborhoods.iloc[dx[i-1]]['Postcode']:
        neighborhoods.iloc[dx[i]]['Neighbourhood'] = neighborhoods.iloc[dx[i-1]]['Neighbourhood'] + ', ' + neighborhoods.iloc[dx[i]]['Neighbourhood']
        neighborhoods.iloc[dx[i-1]]['Neighbourhood'] = np.nan

final_df = neighborhoods.dropna()
final_df = final_df.reset_index(drop=True)
final_df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [8]:
geo = pd.read_csv('http://cocl.us/Geospatial_data')
geo.rename(columns={"Postal Code": "Postcode"}, inplace=True)
geo.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
geo_df = pd.merge(final_df, geo, on='Postcode')
geo_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
